In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt 
import base64
from PIL import Image
import io
import math 
from math import sqrt
import requests
import cv2
import urllib.request
%matplotlib inline

global embed
saved_model_path = 'E:\Graduation Project\Datasets\PRE TRAINED MOBILE NET'
embed = hub.KerasLayer(saved_model_path)

## CONVERTING IMAGES TO FEATURE VECTORS (EMBEDDINGS)

In [4]:
class TensorVector(object):

    def __init__(self, FileName=None):
        self.FileName = FileName

    def process(self):

        img = tf.io.read_file(self.FileName)
        img = tf.io.decode_jpeg(img, channels=3)
        img = tf.image.resize_with_pad(img, 224, 224)
        img = tf.image.convert_image_dtype(img,tf.float32)[tf.newaxis, ...]
        features = embed(img)
        feature_set = np.squeeze(features)
        return list(feature_set)

## function to find the cosine similarity between feature vectors

In [5]:
def cosineSim(a1,a2):
    sum = 0
    suma1 = 0
    sumb1 = 0
    for i,j in zip(a1, a2):
        suma1 += i * i
        sumb1 += j*j
        sum += i*j
    cosine_sim = sum / ((sqrt(suma1))*(sqrt(sumb1)))
    return cosine_sim

## Path Of Main Uploaded Image

In [6]:
logoImgPath = "E:/Graduation Project/Datasets/WebScraping Logos/puna.jpg"

logoImage = cv2.imread(logoImgPath, cv2.IMREAD_ANYCOLOR)  
cv2.imshow("main logo",logoImage)
cv2.waitKey(0)

-1

## getting feature vector of the main logo image

In [7]:
helper = TensorVector(logoImgPath)
vector1 = helper.process()
len(vector1)

1280

## getting feature vector of the retrieved logos images